In [4]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from constants import BTC, BTCTEST, ETH
from pprint import pprint
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account  #decentralized infrastructure for blockchain
from web3.gas_strategies.time_based import medium_gas_price_strategy 
from web3.middleware import geth_poa_middleware

#Middleware is software that lies between an operating system and the applications running on it. 
#Essentially functioning as hidden translation layer, middleware enables communication and data management for distributed applications.

# connect Web3
w3 = Web3(Web3.HTTPProvider('http://localhost:8545')) 

# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

# including a mnemonic with prefunded test tokens for testing
mnemonic = os.getenv("mnemonic")

# Create a function called `derive_wallets`  
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)


# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether") # convert 1.2 ETH to 120000000000 
#below you have to to a function for wei
        gasEstimate = w3.eth.estimateGas({ "to": to, "from": account, "amount": value })
        return {
            "to": to,
            "from": account,
            "value": value,
            "gas": gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce": w3.eth.getTransactionCount(account),
            "chainId": w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])        

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction. YOU NEED TO DIVIDE THIS TO IF IT IS ETH OR IF IT IS BTC *** 
def send_tx(coin, account, to, amount):
    if coin == ETH:
        r_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransaction(r_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        r_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(r_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)

coins = {
    ETH: derive_wallets(coin=ETH),
    BTCTEST: derive_wallets(coin=BTCTEST),
}

pprint(coins)

ModuleNotFoundError: No module named 'constants'